In [28]:
import pandas as pd
import cv2
import numpy as np
import tensorflow as tf
from keras.src.utils.module_utils import tensorflow
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import os

# Charger les données depuis un fichier CSV
data = pd.read_csv(r"C:\Users\33658\PycharmProjects\Act-O-Matic\Datasets\IMDB Scrap Cropped\Natalie Portman Crop\index_cleaned.csv", delimiter=";")

# Charger et prétraiter les images
def preprocess_data(data):
    images, ages = [], []
    base_path = r"C:\Users\33658\PycharmProjects\Act-O-Matic\Datasets\IMDB Scrap Cropped\Natalie Portman Crop"
    for i, row in data.iterrows():
        img_name = str(row["Image"]) + "_face"+ ".jpg" 
        img_path = os.path.join(base_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            print(f"Image non trouvée : {img_path}")
            continue
        img = cv2.resize(img, (224, 224))   # Redimensionner
        img = img / 255.0                   # Normaliser
        images.append(img)
        ages.append(float(row["Age"]))
    return np.array(images), np.array(ages)

# Prétraiter les données
images, ages = preprocess_data(data)

# Diviser en ensembles d'entraînement et de validation
x_train, x_val, y_train, y_val = train_test_split(images, ages, test_size=0.2, random_state=42)

# Convertir en tenseurs
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32).shuffle(1000)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(32)

# Charger le modèle pré-entraîné MobileNetV2
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

# Ajouter des couches personnalisées
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='linear')(x)  # Activation linéaire pour une régression

# Construire le modèle
model = Model(inputs=base_model.input, outputs=output)

# Geler les couches du modèle pré-entraîné
for layer in base_model.layers:
    layer.trainable = False

# Compiler le modèle
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="mean_absolute_error",  # Erreur absolue moyenne pour la régression
    metrics=["mae"]  # Suivi de la MAE
)

# Ajouter un callback pour l'early stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Entraîner le modèle
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=100,
    callbacks=[early_stopping]
)

# Sauvegarder le modèle
model.save("age_estimation_model_Nat.h5")
model.save("../Models/age_estimation_model_Nat.keras")
print("Modèle sauvegardé sous 'age_estimation_model_Nat.h5'")


Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 397ms/step - loss: 23.6520 - mae: 23.6520 - val_loss: 11.0678 - val_mae: 11.0678
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 214ms/step - loss: 9.1877 - mae: 9.1877 - val_loss: 8.0121 - val_mae: 8.0121
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 359ms/step - loss: 8.6109 - mae: 8.6109 - val_loss: 8.4207 - val_mae: 8.4207
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 540ms/step - loss: 8.4269 - mae: 8.4269 - val_loss: 7.0360 - val_mae: 7.0360
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 438ms/step - loss: 6.7240 - mae: 6.7240 - val_loss: 8.1786 - val_mae: 8.1786
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 409ms/step - loss: 7.3862 - mae: 7.3862 - val_loss: 7.3055 - val_mae: 7.3055
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 453ms/step - loss: 6.9125 - mae: 6.9125 - val_loss: 6.7081 - val_mae: 6.7081
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 575ms/step - loss: 7.2463 - mae: 7.2463 - val_loss: 6.6402 - val_mae: 6.6402
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 409ms/step - loss: 5

Modèle sauvegardé sous 'age_estimation_model_Nat.h5'


In [26]:
tf.keras.saving.save_model(model, 'age_estimation_model_cruise.keras')

AttributeError: module 'tensorflow.keras' has no attribute 'saving'

In [27]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Charger le modèle sauvegardé
model = load_model("age_estimation_model_cruise.h5")

# Fonction pour tester avec une image
def predict_age(image_path):
    # Charger l'image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Erreur : Impossible de charger l'image {image_path}")
        return
    
    # Prétraiter l'image (redimensionner et normaliser)
    img = cv2.resize(img, (224, 224))
    img = img / 255.0  # Normalisation
    img = np.expand_dims(img, axis=0)  # Ajouter une dimension pour le batch
    
    # Effectuer la prédiction
    predicted_age = model.predict(img)[0][0]  # Prédiction (régression)
    
    # Afficher le résultat
    print(f"Âge estimé : {predicted_age:.2f} ans")

# Tester avec une image
predict_age(r"C:\Users\33658\Downloads\braff1.jpg")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Âge estimé : 30.66 ans
